Generating plots that explore the disordered LOPC model. This notebook is for testing the disorder method and looking at the optical properties of individual examples of disordered LOPCs without aggregation.

In [ ]:
# computation
import lumapi
import numpy as np
import warnings
import xarray as xr
import xyzpy as xyz
from xyzpy.gen.combo_runner import multi_concat
from multilayer_simulator.lumerical_classes import LumericalOscillator, format_stackrt, format_stackfield
from multilayer_simulator.helpers.mixins import convert_wavelength_and_frequency
import dask
from functools import partial
# plotting
import hvplot.xarray
import holoviews as hv
from holoviews import dim, opts
import panel as pn
import panel.widgets as pnw

In [ ]:
hv.extension("bokeh", "matplotlib", case_sensitive_completion=True)
pn.config.throttled = True
opts.defaults(fontscale=2)

xarray_engine='h5netcdf'

In [ ]:
# A hacky way to import a script

import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
from LOPC import LOPC
from LOPC.helpers import (
    assign_derived_attrs,
    restack,
    enhancement_factor,
    combo_length,
    estimate_combo_run_time,
    linewidth_calculator,
    lopc_data,
    spectrum,
    normalise_over_dim,
    integrate_da,
    sel_or_integrate,
    find_optimum_coords,
    plot_secondary,
    pre_process_for_plots,
    vlines,
    coordinate_string,
    plot_da,
    plot_var,
    plot_optimum_over_dim,
    plot_field,
    visualise_multilayer,
    complex_elements,
    indexer_from_dataset,
    fix_bin_labels,
    mean_and_std,
    max_min_pos,
)

Turn on auto-archiving of cells and Holoviews outputs. See the user guide [here](https://holoviews.org/user_guide/Exporting_and_Archiving.html).

Might need to install `ipympl`.

In [ ]:
# hv.archive.auto() # This breaks the DynamicMap call below due to something to do with pickling

undisordered_run_number = 2
run_number = 111

Useful variables for plotting later:

In [ ]:
blue = hv.Cycle.default_cycles['default_colors'][0]
red = hv.Cycle.default_cycles['default_colors'][1]
yellow = hv.Cycle.default_cycles['default_colors'][2]
green = hv.Cycle.default_cycles['default_colors'][3]

In [ ]:
wavelengths_in_nanometres = np.linspace(480, 880, 256)
wavelengths = wavelengths_in_nanometres * 1e-9
frequencies = convert_wavelength_and_frequency(wavelengths)
angles = np.linspace(0, 86, 64)

In [ ]:
default_oscillator_params = {
    "N": 1e26,
    "permittivity": 2.2,
    "lorentz_resonance_wavelength": 680,
    "lorentz_linewidth": 7.5e13,
}

# Setup

In [ ]:
# fdtd = lumapi.FDTD()

In [ ]:
# oscillator = LumericalOscillator(session=fdtd)

In [ ]:
# rng = np.random.default_rng(seed=20182023)

In [ ]:
# formatter = format_stackrt(
#     output_format="xarray_dataset",
#     **{
#         "variables": ["Rs", "Rp", "Ts", "Tp"],
#         "add_absorption": False,
#     }
# )

In [ ]:
# resources = {
#     "lumerical_session": fdtd,
#     "oscillator": oscillator,
#     "rng": rng,  # assign seeded rng for reproducibility
#     "simulation_mode": "stackrt",
#     "formatter": formatter,
#     "frequencies": frequencies,
#     "angles": angles,
# }

In [ ]:
# constants = {
#     "apply_disorder": 10,  # True-> only one run; int-> int runs
#     "delta_mode": "pplt",  # abs-> interpret delta as distance in nm; pplt-> interpret delta as proportion of passive layer thickness
#     "disorder_type": "uniform",  # not considering gaussian or other distributions
#     "correlated": False,  # False-> uncorrelated disorder
#     "retain_runs": False,  # True-> return concat'd array; False-> return mean array
#     "add_first_layer": True,  # add dummy passive layer to allow disorder on first excitonic layer
#     "copy_layers": True,
#     "length_scale": 1e-9,
# }

In [ ]:
# r = xyz.Runner(lopc_data, var_names=None, constants=constants, resources=resources)

## LOPC

In [ ]:
# h_lopc = xyz.Harvester(runner=r, data_name=f"data/run_{run_number}/LOPC.nc")

In [ ]:
# %%time
# # check how long one run takes
# lopc_data(**resources, **constants)
# # 9s

## Reference slabs

The reference slabs are just those from the equivalent undisordered run.

# Testing

## Perturbing

In [ ]:
from LOPC.helpers import visualise_multilayer, thicknesses_to_vspans

In [ ]:
from LOPC.LOPC import adjust_layer_thickness, shift_layer_position

In [ ]:
# expect a warning
lopc = LOPC.LOPC(**resources, **constants, passive_layer_thickness=200, excitonic_layer_thickness=50, remove_last_layer=False)

In [ ]:
lopc.structure.layers

In [ ]:
p_layer = lopc.structure.layers[2]

In [ ]:
adjust_layer_thickness(p_layer, 1e-7)

In [ ]:
p_layer

In [ ]:
e_layer = lopc.structure.layers[1]

In [ ]:
adjust_layer_thickness(e_layer, 5e-8)

In [ ]:
e_layer

In [ ]:
visualise_multilayer(lopc.structure).opts(opts.VSpan(apply_ranges=True, color=blue, width=400))

In [ ]:
shift_layer_position(lopc.structure, layer_index=7, delta=1e-7, compensate=True)

In [ ]:
visualise_multilayer(lopc.structure).opts(opts.VSpan(apply_ranges=True, color=blue, width=400))

In [ ]:
shift_layer_position(lopc.structure, layer_index=11, delta=1e-7, compensate=False)

In [ ]:
visualise_multilayer(lopc.structure).opts(opts.VSpan(apply_ranges=True, color=blue, width=400))

In [ ]:
shift_layer_position(lopc.structure, layer_index=13, delta=-4e-7, compensate=True)

In [ ]:
visualise_multilayer(lopc.structure).opts(opts.VSpan(apply_ranges=True, color=blue, width=400))

In [ ]:
lopc.structure.layers

In [ ]:
excitonic_layers = lopc.structure.layers[1:-1:2]
excitonic_layers

In [ ]:
list(range(1, len(lopc.structure.layers)-1, 2))

In [ ]:
lopc.structure.layers[20]

In [ ]:
a = not 0

In [ ]:
a

## Random

In [ ]:
np.random.seed(123)
np.random.rand()  # deprecated

In [ ]:
rng = np.random.default_rng(123)
rng.random()

In [ ]:
rng = np.random.default_rng(123)
np.random.default_rng(rng).random()  # test: passing a generator to .default_rng should return the same generator

In [ ]:
rng == np.random.default_rng(rng)

In [ ]:
rng == np.random.default_rng(123)

In [ ]:
rng2 = np.random.default_rng(123)

In [ ]:
rng.random(10)[:-1] == rng2.random(10)[1:]  # the next 9 outputs of the first generator are the 2nd-10th output of the fresh one

In [ ]:
%%timeit

[rng.uniform(-1, 1) for _ in range(0, 10)]

In [ ]:
%%timeit

rng.uniform(-1, 1, 10)

In [ ]:
[rng.uniform(-1, 1) for _ in range(0, 10)]

In [ ]:
rng.uniform(-1, 1, 10)

# Data generation

In [ ]:
deltas = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15]#, 0.2, 0.25, 0.3, 0.4, 0.5]

In [ ]:
combos = {
    "passive_RI": [1.35],
    "incident_medium_RI": [1.35],
    "exit_medium_RI": [1.35],
    "N": [1e26],
    "permittivity": [2.2],
    "lorentz_resonance_wavelength": [680],
    "lorentz_linewidth": [7.5e13],
    "num_periods": np.arange(5, 30, 5),
    "passive_layer_thickness": np.arange(150, 301, 10),
    "excitonic_layer_thickness": np.arange(10, 81, 10),
    "remove_last_layer": [0], # True/False are not compatible with netCDF format
}

In [ ]:
combos_with_delta = {d: combos|{"delta": [d]} for d in deltas}

In [ ]:
# combos_with_delta[0.1]

## Layer thicknesses

### LOPC simulation

In [ ]:
# for k, v in combos_with_delta[0.1].items():
#     print(f"{k}: length {len(v)}")

In [ ]:
# estimate_combo_run_time(9, combos)/3600

In [ ]:
# deltas used (in order, by value): [0.1, 0.01, 0.02, 0.03, 0.04]
# seed: 20182023

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     h_lopc.harvest_combos(combos_with_delta[0.1])

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     h_lopc.harvest_combos(combos_with_delta[0.01])

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     h_lopc.harvest_combos(combos_with_delta[0.02])

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     h_lopc.harvest_combos(combos_with_delta[0.03])

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     h_lopc.harvest_combos(combos_with_delta[0.04])

In [ ]:
# h_lopc.last_ds.to_netcdf(path=f"data/run_{run_number}/LOPC_004.nc", engine=xarray_engine, invalid_netcdf=True)

In [ ]:
# deltas used (in order, by value): [0.05, 0.06, 0.07, 0.08, 0.09]
# seed: 20182023106-20182023110

In [ ]:
# rngs = [np.random.default_rng(seed=20182023102+i) for i in range(4, 9)]

In [ ]:
[deltas[i] for i in range(4,9)]

In [ ]:
# runners = [
#     xyz.Runner(
#         lopc_data,
#         var_names=None,
#         constants=constants,
#         resources=resources | {"rng": rngs[i]},
#     )
#     for i in range(5)
# ]

In [ ]:
# harvesters = [xyz.Harvester(runner=runners[i], data_name=f"data/run_{run_number}/LOPC_00{i+5}.nc") for i in range(5)]

In [ ]:
combos_with_delta[deltas[1+4]]

In [ ]:
# # only completed the first loop
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     for i in range(5):
#         harvesters[i].harvest_combos(combos_with_delta[deltas[i+4]])

In [ ]:
# # loops 2-5
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     for i in range(1, 5):
#         harvesters[i].harvest_combos(combos_with_delta[deltas[i+4]])

In [ ]:
# deltas used (in order, by value): [0.11, 0.12, 0.13, 0.14, 0.15]
# seed: 20182023112-20182023116

In [ ]:
# rngs = [np.random.default_rng(seed=20182023102+i) for i in range(10, 15)]

In [ ]:
# runners = [
#     xyz.Runner(
#         lopc_data,
#         var_names=None,
#         constants=constants,
#         resources=resources | {"rng": rngs[i]},
#     )
#     for i in range(5)
# ]

In [ ]:
# harvesters = [xyz.Harvester(runner=runners[i], data_name=f"data/run_{run_number}/LOPC_01{i+1}.nc") for i in range(5)]

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     for i in range(5):
#         harvesters[i].harvest_combos(combos_with_delta[deltas[i+10]])

How embarrasing, I forgot that `np.arange(5, 30, 5)` doesn't include 30 itself, so I need to do those runs separately:

In [ ]:
# rngs = [np.random.default_rng(seed=201820231111+i) for i in range(len(deltas))]

In [ ]:
# runners = [
#     xyz.Runner(
#         lopc_data,
#         var_names=None,
#         constants=constants,
#         resources=resources | {"rng": the_rng},
#     )
#     for the_rng in rngs
# ]

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "Can't deepcopy")
#     for r, d in zip(runners, deltas):
#         h_lopc = xyz.Harvester(runner=r, data_name=f"data/run_{run_number}/LOPC_N30_{str(d).replace('.', '')}.nc")
#         h_lopc.harvest_combos(combos|{"num_periods": [30], "delta": [d]})

In [ ]:
# ds = h_lopc.full_ds.copy()

### Load LOPC dataset

In [ ]:
# # chunks for per-angle plots
# chunks = {
#     "frequency": 256,
#     "excitonic_layer_thickness": 16,
#     "passive_layer_thickness": 32,
#     "theta": 1,
#     "num_periods": 16,
# }

In [ ]:
# chunks for plotting or integrating over angle
chunks = {
    "frequency": 256,
    "excitonic_layer_thickness": 8,
    "passive_layer_thickness": 16,
    "theta": 16,
    "num_periods": 5,
    "delta": 1,
}

#### Maths to work out good chunk sizes

In [ ]:
def chunk_size(chunks):
    return np.prod(list(chunks.values()))

In [ ]:
# Number of chunks
(combo_length(combos)*256*64) / chunk_size(chunks)

In [ ]:
# Do chunks contain at least a million data points?
chunk_size(chunks)/1e6

####

In [ ]:
# ds_short = xr.open_mfdataset(
#     [
#         f"data/run_{run_number}/LOPC.nc",
#         f"data/run_{run_number}/LOPC_004.nc",
#         f"data/run_{run_number}/LOPC_005.nc",
#         f"data/run_{run_number}/LOPC_006.nc",
#         f"data/run_{run_number}/LOPC_007.nc",
#         f"data/run_{run_number}/LOPC_008.nc",
#         f"data/run_{run_number}/LOPC_009.nc",
#         f"data/run_{run_number}/LOPC_011.nc",
#         f"data/run_{run_number}/LOPC_012.nc",
#         f"data/run_{run_number}/LOPC_013.nc",
#         f"data/run_{run_number}/LOPC_014.nc",
#         f"data/run_{run_number}/LOPC_015.nc",
#     ],
#     concat_dim="delta",
#     combine="nested",
#     engine=xarray_engine,
#     lock=False,
#     chunks=chunks,
# )

In [ ]:
# ds_long = xr.open_mfdataset(
#     f"data/run_{run_number}/LOPC_N30_*.nc",
#     concat_dim="delta",
#     combine="nested",
#     engine=xarray_engine,
#     lock=False,
#     chunks=chunks,
# )

In [ ]:
# # sort delta values and save as one aggregated file
# ds = xr.concat([ds_short.sortby("delta"), ds_long], dim="num_periods")
# ds.to_netcdf(f"data/run_199/LOPC_aggregated.nc", engine=xarray_engine)

Note: to save disk space, all generated data in `run_111` has been deleted. It's all in `run_199` now anyway.

In [ ]:
ds = xr.open_mfdataset(
    f"data/run_199/LOPC_aggregated.nc",    
    engine=xarray_engine,
    lock=False,
    chunks=chunks,
)

In [ ]:
indexer_from_disorder_ds = partial(indexer_from_dataset, drop=["delta"])

In [ ]:
periodic_ds = xr.open_mfdataset(
    f"data/run_{undisordered_run_number}/LOPC.nc",
    engine=xarray_engine,
    lock=False,
    chunks=chunks,
)

In [ ]:
# loading the data into memory saves times later
pds = periodic_ds.sel(indexer_from_disorder_ds(ds)).load()

In [ ]:
ds = xr.concat([pds.expand_dims(delta = [0.0], axis=-3), ds], dim="delta")

In [ ]:
# add derived attrs
ds = assign_derived_attrs(ds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"])
# ds

In [ ]:
restack_plt_to_period = partial(
    restack,
    start_idxs=["passive_layer_thickness", "excitonic_layer_thickness"],
    end_idxs=["period", "excitonic_layer_thickness"],
)

In [ ]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
restacked_ds = restack_plt_to_period(ds)

In [ ]:
# add derived attrs
pds = assign_derived_attrs(pds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"])

In [ ]:
restacked_pds = restack_plt_to_period(pds)

### Load reference slab dataset

In [ ]:
# useful variables
total_excitonic_thicknesses = np.unique(ds.total_excitonic_thickness)
total_passive_thicknesses = np.unique(ds.total_passive_thickness)
total_thicknesses = np.unique(ds.total_thickness)

In [ ]:
ref = xr.open_mfdataset(
    f"data/run_{undisordered_run_number}/ref.nc",
    engine=xarray_engine,
    lock=False,
    # chunks={'theta': 16},
)

Note: `period=False` is an important option because otherwise it ends up a coordinate of `total_excitonic_thickness` and causes a conflict after binary operations with `ds`.

In [ ]:
ref = assign_derived_attrs(ref, period=False, total_excitonic_thickness=False, total_passive_thickness=False, total_thickness=False)
# ref

In [ ]:
# compressed reference slab without passive layer
crs_1 = (
    ref.sel(
        remove_last_layer=1,
        passive_layer_thickness=0,
        excitonic_layer_thickness=total_excitonic_thicknesses,
        drop=True,
    )
    .squeeze(drop=True)
    .rename(excitonic_layer_thickness="total_excitonic_thickness")
    .load()
)

# # compressed reference slab with the passive layer at the end
# crs_0 = (
#     ref.sel(
#         remove_last_layer=0,
#         passive_layer_thickness=total_passive_thicknesses,
#         excitonic_layer_thickness=total_excitonic_thicknesses,
#         drop=True,
#     )
#     .squeeze(drop=True)
#     .rename(
#         excitonic_layer_thickness="total_excitonic_thickness",
#         passive_layer_thickness="total_passive_thickness",
#     )
# )

### Calculate derived variables

#### Normalise by CRS

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    norm_1 = enhancement_factor(ds, ref=crs_1, common_dim="total_excitonic_thickness", method="groupby")

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    restacked_norm_1 = restack_plt_to_period(norm_1)

#### Difference with CRS

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    gb_tet = ds.groupby('total_excitonic_thickness')

In [ ]:
# 'biology' absorptance enhancement factor: normalise by reference slab type 1: compressed reference slab w/o passive layer
diff_1 = gb_tet - crs_1

In [ ]:
# diff_1 = assign_derived_attrs(
#     dataset=diff_1,
#     unpolarised=True,
#     absorption=False,
#     period=False,
#     total_excitonic_thickness=False,
#     total_passive_thickness=False,
#     total_thickness=False,
#     N_tot=False,
#     per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
# )

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    restacked_diff_1 = restack_plt_to_period(diff_1)

# Plots

In [ ]:
unpolarised_RTA = ["R", "T", "A"]
s_polarised_RTA = ["Rs", "Ts", "As"]
reflectance = ["Rs", "Rp", "R"]
transmittance = ["Ts", "Tp", "T"]
absorptance = ["As", "Ap", "A"]
per_oscillator_RTA = ["R_per_oscillator", "T_per_oscillator", "A_per_oscillator"]

In [ ]:
# WARNING: all these datasets will be fundamentally changed after this cell, to the extent that it can't be run twice
# For consistency, keep important calculations in the preceding section!
ds = pre_process_for_plots(ds)
restacked_ds = pre_process_for_plots(restacked_ds)
pds = pre_process_for_plots(pds)
restacked_pds = pre_process_for_plots(restacked_pds)
ref = pre_process_for_plots(ref)
crs_1 = pre_process_for_plots(crs_1)
norm_1 = pre_process_for_plots(norm_1)
restacked_norm_1 = pre_process_for_plots(restacked_norm_1)
diff_1 = pre_process_for_plots(diff_1)
restacked_diff_1 = pre_process_for_plots(restacked_diff_1)

In [ ]:
for c in ds.coords.values():
    print(c.attrs)

In [ ]:
period_dim = hv.Dimension("period", label="Λ", unit="nm")
wavelength_dim = hv.Dimension("wavelength", label="λ", unit="nm")
real_index_dim = hv.Dimension("n")
imag_index_dim = hv.Dimension("k", label="ϰ")

### Useful lines

#### Function to do linewidth maths

In [ ]:
def linewidth_calculator_factory(centre, linewidth):
    return partial(linewidth_calculator, centre=centre, linewidth=linewidth)

#### Lorentz lines

I want some sort of metric for 'near the resonance' and 'far from the resonance'. The natural unit of distance in this instance is the linewidth. The linewidth is given in rad/s so there need to be some conversions to get the equivalent lines in the plots by wavelength, but they are roughly symmetrical around the peak wavelength.

Based on the plots of the refractive index below, I think I will consider 'near' to be 'within two linewidths', and 'far' to be 'at least four linewidths away'.

In [ ]:
# resonance_line = hv.VLine(680, label='LO resonance wavelength').opts(line_dash='dotted')

# Convert from rad/s to Hz
lorentz_linewidth_frequency = default_oscillator_params["lorentz_linewidth"] / (2*np.pi)

In [ ]:
lorentz_line_frequency = linewidth_calculator_factory(convert_wavelength_and_frequency(680e-9), lorentz_linewidth_frequency)

def lorentz_line_wavelength(x=None):
    x = -x if x is not None else x
    return convert_wavelength_and_frequency(lorentz_line_frequency(x))

In [ ]:
def lorentz_vlines(x=0, scale=1, mode='wavelength', **kwargs):
    if mode == 'wavelength':
        line_func = lorentz_line_wavelength
    elif mode == 'frequency':
        line_func = lorentz_line_frequency
    else:
        raise TypeError(f"mode should be 'wavelength' or 'frequency', not {mode}")
        
    match x:
        case [*xs]:
            line_pos = [line_func(x)/scale for x in xs]
        case x:
            line_pos = line_func(x)/scale
            
    return vlines(line_pos, **kwargs)

### Useful functions

#### Select a wavelength or wavelength range based on the distance from the resonance in linewidths.

In [ ]:
def select_lorentz_line(da, lorentz_line=0, window_radius=0):
    if window_radius == 0:
        wavelength = lorentz_line_wavelength(lorentz_line) * 1e9
        wavelength_sel_method = "nearest"
    else:
        wavelength = slice(
            lorentz_line_wavelength(lorentz_line - window_radius) * 1e9,
            lorentz_line_wavelength(lorentz_line + window_radius) * 1e9,
        )
        wavelength_sel_method = None
    da = da.sel(wavelength=wavelength, method=wavelength_sel_method)
    
    return da

#### Plot a comparison of the reflectance and absorptance of the LOPC with that of the reference slab.

In [ ]:
opts_R = [opts.Curve(color=blue, ylim=(0,1)), opts.Image(cmap='viridis', clim=(0,1)), opts.QuadMesh(cmap='viridis', clim=(0,1))]

def plot_R(variable="R", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_R)
    return plot

# # test
# plot_R(dataset=restacked_ds.sel(period=200, excitonic_layer_thickness=50, num_periods=10).squeeze(), x="wavelength", y="theta")

In [ ]:
opts_T = [opts.Curve(color=yellow, ylim=(0,1)), opts.Image(cmap='cividis', clim=(0,1)), opts.QuadMesh(cmap='cividis', clim=(0,1))]

def plot_T(variable="T", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_T)
    return plot

In [ ]:
opts_A = [opts.Curve(color=red, ylim=(0,1)), opts.Image(cmap='inferno', clim=(0,1)), opts.QuadMesh(cmap='inferno', clim=(0,1))]

def plot_A(variable="A", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_A)
    return plot

In [ ]:
def plot_vars_to_funcs(plot_vars):
    var_func_mapping = {
        "R": plot_R,
        "T": plot_T,
        "A": plot_A,
        "Rs": partial(plot_R, variable="Rs"),
        "Ts": partial(plot_T, variable="Ts"),
        "As": partial(plot_A, variable="As"),
        "Rp": partial(plot_R, variable="Rp"),
        "Tp": partial(plot_T, variable="Tp"),
        "Ap": partial(plot_A, variable="Ap"),
        "R_per_oscillator": partial(plot_R, variable="R_per_oscillator"),
        "T_per_oscillator": partial(plot_T, variable="T_per_oscillator"),
        "A_per_oscillator": partial(plot_A, variable="A_per_oscillator"),
        "Rs_per_oscillator": partial(plot_R, variable="Rs_per_oscillator"),
        "Ts_per_oscillator": partial(plot_T, variable="Ts_per_oscillator"),
        "As_per_oscillator": partial(plot_A, variable="As_per_oscillator"),
        "Rp_per_oscillator": partial(plot_R, variable="Rp_per_oscillator"),
        "Tp_per_oscillator": partial(plot_T, variable="Tp_per_oscillator"),
        "Ap_per_oscillator": partial(plot_A, variable="Ap_per_oscillator"),
    }

    plot_funcs = []
    for var in plot_vars:
        try:
            func = var_func_mapping[var]
        except KeyError:
            func = partial(plot_var, variable=var)
        plot_funcs.append(func)

    return plot_funcs

In [ ]:
# new version
def plot_RTA(
    period,
    excitonic_layer_thickness,
    num_periods,
    theta,
    delta,
    title="",
    include=["LOPC", "CRS_1"],
    plot_vars=["R", "T", "A"],
    label_override=None,
    label_append=None,
):
    label_field = None  # for debugging
    label_append = "" if label_append is None else label_append

    P = period
    t = excitonic_layer_thickness
    N = num_periods
    D = delta

    plot_funcs = [
        partial(func, x="wavelength", label_field=label_field)
        for func in plot_vars_to_funcs(plot_vars)
    ]
    curves = []
    if "LOPC" in include:
        lopc_label = " (LOPC)" if label_override is None else label_override
        lopc_label += label_append
        lopc_sel = restacked_ds.sel(
            period=P, excitonic_layer_thickness=t, num_periods=N, delta=D
        ).squeeze()
        lopc_sel = sel_or_integrate(lopc_sel, "theta", theta, normalisation=1)
        lopc_curves = [
            plot_func(dataset=lopc_sel, label_append=lopc_label).opts(line_dash="solid")
            for plot_func in plot_funcs
        ]
        curves += lopc_curves
    if "CRS_1" in include:
        crs_1_label = " (CRS)" if label_override is None else label_override
        crs_1_label += label_append
        crs_1_sel = crs_1.sel(total_excitonic_thickness=t * N).squeeze()
        crs_1_sel = sel_or_integrate(crs_1_sel, "theta", theta, normalisation=1)
        crs_1_curves = [
            plot_func(dataset=crs_1_sel, label_append=crs_1_label).opts(
                line_dash="dashed"
            )
            for plot_func in plot_funcs
        ]
        curves += crs_1_curves
    if "FRS_1" in include:
        frs_1_label = " (FRS)" if label_override is None else label_override
        frs_1_label += label_append
        frs_1_sel = frs_1.sel(total_thickness=(P + t) * N).squeeze()
        frs_1_sel = sel_or_integrate(frs_1_sel, "theta", theta, normalisation=1)
        frs_1_curves = [
            plot_func(dataset=frs_1_sel, label_append=frs_1_label).opts(
                line_dash="dotted"
            )
            for plot_func in plot_funcs
        ]
        curves += frs_1_curves

    overlay = hv.Overlay(curves).opts(
        opts.Curve(
            ylim=(0, 1),
            ylabel="Intensity",
            title=f"{title}{coordinate_string(period=P, excitonic_layer_thickness=t, num_periods=N, theta=theta, delta=D)}",
        ),
    )

    return overlay


# # test
display(
    plot_RTA(200, 50, 20, 0, 0.15, "test\n", include=["LOPC", "CRS_1"]).opts(
        legend_position="right"
    )
)

# display(
#     plot_RTA(
#         200,
#         50,
#         20,
#         (10, 50),
#         "test RA only\n",
#         include=["LOPC", "CRS_1"],
#         plot_vars=["R", "A"],
#     ).opts(opts.Overlay(legend_position="right"))
# )

# display(
#     plot_RTA(
#         200,
#         50,
#         20,
#         75,
#         "test\n",
#         include=["LOPC"],
#         plot_vars=["R_per_oscillator", "A_per_oscillator"],
#         label_append=" test",
#         label_override="OVERRIDDEN",
#     ).opts(opts.Curve(ylim=(None, None)), opts.Overlay(legend_position="right"))
# )

#### Plot a comparison of normal incidence to integrated

In [ ]:
def plot_comparison(*comparison_params: tuple[dict, list["opts"]], plot_func=plot_RTA, **shared_params):
    param_opts = [(shared_params|comp_params, comp_opts) for comp_params, comp_opts in comparison_params]
    plots = [plot_func(**comp_params).opts(*comp_opts) for comp_params, comp_opts in param_opts]
    return plots

In [ ]:
def compare_RTA(*args, opts_cycle=None, **shared_params):
    default_opts = [[opts.Curve(line_dash=style)] for style in ["solid", "dashed", "dotted", "dotdash", "dashdot"]]
    opts_cycle = default_opts if opts_cycle is None else opts_cycle
    
    # comparison_params = list(zip(args, opts_cycle))
    
    plots = plot_comparison(*zip(args, opts_cycle), plot_func=plot_RTA, **shared_params)
    overlay = hv.Overlay(plots).opts(opts.Overlay(legend_position="right"))
                                     
    return overlay

# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
#     "include": ["LOPC"],
# }
# compare_RTA({"theta": (0, 75), "label_override": " (integrated)"}, {"theta": 0, "label_override": " (θ = 0)"}, **shared_params)

In [ ]:
compare_RTA_normal_vs_integrated = partial(compare_RTA, {"theta": (0, 45), "label_override": " (integrated)"}, {"theta": 0, "label_override": " (θ = 0)"}, include= ["LOPC"],)

# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
# }
# compare_RTA_normal_vs_integrated(**shared_params)

#### Plot the RTA of the structures in 2D

In [ ]:
def plot_RTA_2D(
    period,
    excitonic_layer_thickness,
    num_periods,
    theta=(0, 75),
    delta=0,
    title="",
    include=["LOPC", "CRS_1"],
):
    P = period
    t = excitonic_layer_thickness
    N = num_periods
    D = delta

    plots = []
    if "LOPC" in include:
        lopc_sel = restacked_ds.sel(
            period=P, excitonic_layer_thickness=t, num_periods=N, delta=D
        ).squeeze()
        lopc_sel = lopc_sel.sel(theta=slice(*theta))
        plots.append(
            lopc_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (LOPC)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            lopc_sel["T"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Transmittance (LOPC)")
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            lopc_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (LOPC)")
            .opts(opts.Image(cmap="inferno"))
        )
    if "CRS_1" in include:
        crs_1_sel = crs_1.sel(total_excitonic_thickness=t * N).squeeze()
        crs_1_sel = crs_1_sel.sel(theta=slice(*theta))
        plots.append(
            crs_1_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (CRS)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            crs_1_sel["T"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Transmittance (CRS)")
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            crs_1_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (CRS)")
            .opts(opts.Image(cmap="inferno"))
        )
    if "FRS_1" in include:
        frs_1_sel = frs_1.sel(total_thickness=(P + t) * N).squeeze()
        frs_1_sel = frs_1_sel.sel(theta=slice(*theta))
        plots.append(
            frs_1_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (FRS)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            frs_1_sel["T"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Transmittance (FRS)")
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            frs_1_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (FRS)")
            .opts(opts.Image(cmap="inferno"))
        )

    layout = hv.Layout(plots).opts(
        opts.Image(
            clim=(0, 1),
            clabel="Intensity",
        ),
        opts.Layout(
            title=f"{title}{coordinate_string(period=P, excitonic_layer_thickness=t, num_periods=N, delta=D)}",
        ),
    )

    return layout

# # test
display(plot_RTA_2D(200, 50, 20, (0, 90), 0.15, "test\n", include=["LOPC", "CRS_1"]).opts(opts.Image(frame_width=200)).cols(3))

# display(plot_RTA_2D(200, 40, 20, (10, 50), "test\n", include=["LOPC", "CRS_1", "FRS_1"]).opts(opts.Image(frame_width=200)).cols(3))

#### Plot an enhancement factor.

In [ ]:
def plot_ef(
    variable,
    dataset,
    sel=None,
    sel_method=None,
    title="",
    *,
    x="wavelength",
    y=None,
):
    sel = {} if sel is None else sel
    da = dataset[variable].sel(**sel, method=sel_method).squeeze()
    if y is None:
        plot = da.hvplot(x=x, label=f"{variable} enhancement factor")
        plot *= hv.HLine(1).opts(line_dash="dotted")
    else:
        plot = da.hvplot(
            kind="image",
            x=x,
            y=y,
            label=f"{variable} enhancement factor",
            clim=(0.5, 1.5),
        )
    plot = plot.opts(
        opts.Curve(
            title=f"{title}{coordinate_string(**sel)}",
        ),
        opts.Overlay(
            title=f"{title}{coordinate_string(**sel)}",
        ),
    )

    return plot


# # test
sel_1 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 30}
sel_2 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 0}
sel_3 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10}
display(
    (
        plot_ef("As", restacked_norm_1, sel_1, "nearest", "test\n")
        # + plot_ef("As", restacked_norm_2, sel_2, title="test2\n")
    )#.cols(1)
)
display(
    plot_ef("As", restacked_norm_1, sel_3, title="test3\n", x="theta", y="wavelength").opts(clim=(0, 2), cmap="RdBu_r")
)

#### Test plot_optimum_over_dim

In [ ]:
foo, bar = plot_optimum_over_dim(restacked_ds.A.sel(theta=0, wavelength=660, method="nearest"), "period", "excitonic_layer_thickness", "num_periods", "max")

In [ ]:
foo, bar = plot_optimum_over_dim(integrate_da(restacked_ds.A, "theta", normalisation=1).sel(wavelength=660, method="nearest"), ["period", "delta"], "excitonic_layer_thickness", "num_periods", "max")

In [ ]:
foo, bar = plot_optimum_over_dim(integrate_da(restacked_ds.R, "wavelength", normalisation=1).sel(theta=0, method="nearest"), ["period", "excitonic_layer_thickness"], "num_periods", "delta", "max")

In [ ]:
foo

In [ ]:
print(foo.Quadmesh[25])

In [ ]:
hv.Overlay([hv.Curve((foo.QuadMesh[N].dimension_values(1), foo.QuadMesh[N].dimension_values(2))) for N in foo.QuadMesh.dimension_values(0)])

In [ ]:
bar

#### Find and plot the min or max over any dimension.

In [ ]:
def wrapped_2D_plot(
    variable,
    dataset,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=None,
    period_stop=None,
    integrate_angle=None,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    dim=None,  # automatically assign if dataset recognised
):
    plots = []

    if str(dataset) == "restacked_ds":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_ds.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    if str(dataset) == "restacked_norm_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_norm_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    if str(dataset) == "restacked_diff_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_diff_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    da = dataset[variable]

    if not integrate_angle:
        da = da.sel(theta=theta, method="nearest")
    else:  # integrate_angle must be a float, so that (theta, integrate_angle) is a slice syntax
        da = da.sel(theta=slice(theta, integrate_angle))
        da = integrate_da(da, "theta", weighting=1, normalisation=1)

    if period_start < period_stop:
        da = da.sel(period=slice(period_start, period_stop))
    else:  # otherwise no data is selected and everything breaks
        da = da.sel(period=slice(period_start, None))
    da = select_lorentz_line(da, lorentz_line=lorentz_line, window_radius=window_radius)

    vline_locs = [0]

    if window_radius == 0:
        wavelength = float(da.wavelength)
        title = f"{optimise.capitalize()}imum {variable} at {wavelength:.0f} nm"
        if lorentz_line != 0:  # don't put two lines over the resonance
            vline_locs.append(lorentz_line)
    else:
        wavelength_start = float(da.wavelength[0])
        wavelength_stop = float(da.wavelength[-1])
        # make it easier to compare values
        da = integrate_da(
            da, "wavelength", weighting=1, normalisation=1
        )  # replaces the below two lines
        # da = normalise_over_dim(da, "wavelength", 1)
        # da = da.integrate("wavelength")
        title = f"{optimise.capitalize()}imum integrated {variable} between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
        vline_locs.append(lorentz_line - window_radius)
        vline_locs.append(lorentz_line + window_radius)

    plot_1, optimum_coords = plot_optimum_over_dim(
        da,
        dim=dim,
        x="excitonic_layer_thickness",
        y="num_periods",
        optimise=optimise,
    )

    P = float(optimum_coords["period"])
    t = float(optimum_coords["excitonic_layer_thickness"])
    N = float(optimum_coords["num_periods"])
    D = float(optimum_coords["delta"])
    try:  # this should work if not integrating over theta
        th = float(optimum_coords["theta"])
    except:  # probably the problem is that theta doesn't exist because I integrated over it already
        th = (theta, integrate_angle)
    lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
        opts.VLine(line_color=green, line_dash="dotted"),
    )

    # give the resonance line a special colour
    lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

    plot_1.opts(
        opts.QuadMesh(cmap=cmap),
        opts.Points(color="red"),
        opts.Overlay(
            title=f"{title}\nOptimal period: {P:.0f}\nOptimal Delta: {D:.2f}"
        ),
    )

    plots.append(plot_1)

    if "RTA_normal" in extra_plots:  # plot RTA at theta=0
        new_plot = plot_RTA(
            period=P,
            excitonic_layer_thickness=t,
            num_periods=N,
            theta=0,
            delta=D,
        )
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
        new_plot = plot_RTA(
            period=P,
            excitonic_layer_thickness=t,
            num_periods=N,
            theta=th,
            delta=D,
        )
        new_plot *= lorentz_lines
        plots.append(new_plot)

    if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
        sel = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
            "theta": 0,
            "delta": D,
        }
        new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if (
        "norm_1_int" in extra_plots
    ):  # plot enhancement factor at theta OR integrating over theta
        try:  # this should work if not integrating over theta
            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "theta": th,
                "delta": D,
            }
            new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
            new_plot *= lorentz_lines
        except:  # if integrating, we need to do the integral *before* normalising
            ds_int = sel_or_integrate(ds, dim="theta", val=th)
            crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
            norm = enhancement_factor(
                ds_int,
                ref=crs_1_int,
                common_dim="total_excitonic_thickness",
                method="groupby",
            )
            restacked_norm = restack_plt_to_period(norm)
            # replaces the lines below
            #             # this should all get separated out into its own function
            #             crs_1_like_ds = crs_1.sel(
            #                 total_excitonic_thickness=ds.total_excitonic_thickness
            #             )

            #             ds_int = sel_or_integrate(ds, dim="theta", val=th)
            #             crs_1_int = sel_or_integrate(crs_1_like_ds, dim="theta", val=th)
            #             norm = ds_int / crs_1_int
            #             restacked_norm = norm.stack(multiperiod=['passive_layer_thickness', 'excitonic_layer_thickness']).set_index(multiperiod=['period', 'excitonic_layer_thickness']).unstack()

            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "delta": D,
            }
            new_plot = plot_ef(variable="A", dataset=restacked_norm, sel=sel)
            new_plot *= lorentz_lines
            sel["theta"] = th
            new_plot = new_plot.opts(opts.Overlay(title=f"{coordinate_string(**sel)}"))
        plots.append(new_plot)

    return hv.Layout(plots).cols(1)

In [ ]:
wrapped_2D_plot(
    "A_per_oscillator",
    restacked_ds,
    optimise="max",
    lorentz_line=5,
    window_radius=2,
    theta=0,
    cmap="inferno",
    period_start=0,
    period_stop=1000,
    integrate_angle=None,
    extra_plots=["RTA_int", "norm_1_int"],
    dim="period",  # automatically assign if dataset recognised
)

#### Plot the E-field, overlayed with the refractive index profile and layer boundaries.

In [ ]:
# sometimes this errors on the first call for some reason
fdtd = lumapi.FDTD()

In [ ]:
oscillator = LumericalOscillator(fdtd)

In [ ]:
plot_field(
    680,
    lumerical_session=fdtd,
    oscillator=oscillator,
    ri_lower=1.35,
    ri_upper=1.6,
    excitonic_layer_thickness=30,
    passive_layer_thickness=210,
    num_periods=10,
    copy_layers=True,
    apply_disorder=True,
    delta=0.5,
    delta_mode="pplt",  # abs-> interpret delta as distance in nm; pplt-> interpret delta as proportion of passive layer thickness
    disorder_type="uniform",  # not considering gaussian or other distributions
    correlated=True,  # False-> uncorrelated disorder
    retain_runs=True,  # True-> return concat'd array; False-> return mean array
    add_first_layer=True,  # add dummy passive layer to allow disorder on first excitonic layer
).opts(opts.VSpan(color="gray"))

In [ ]:
def wrap_plot_field(
    wavelength, excitonic_layer_thickness, passive_layer_thickness, num_periods
):
    coords = {
        "λ": wavelength,
        "Excitonic layer thickness": excitonic_layer_thickness,
        "Passive layer thickness": passive_layer_thickness,
        "Number of periods": num_periods,
    }

    title = f"{coordinate_string(**coords)}"

    return plot_field(
        wavelength=wavelength,
        lumerical_session=fdtd,
        oscillator=oscillator,
        ri_lower=1.35,
        ri_upper=1.6,
        excitonic_layer_thickness=excitonic_layer_thickness,
        passive_layer_thickness=passive_layer_thickness,
        num_periods=num_periods,
    ).opts(opts.Curve(title=title, ylim=(0,None)), opts.VSpan(color="gray"))

In [ ]:
# an example of what this can do
pn.interact(
    wrap_plot_field,
    wavelength=(480, 880),
    excitonic_layer_thickness=(10, 200),
    passive_layer_thickness=(0, 300),
    num_periods=(1, 50),
)

# Miscellanous stuff

In [ ]:
gb = restacked_ds.groupby_bins("run", 2, labels=range(2))

In [ ]:
rds_mean = (gb.mean("run"))

In [ ]:
xs_1D = {
    # "delta": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "period": pnw.DiscreteSlider,
    "excitonic_layer_thickness": pnw.DiscreteSlider,
}

rds_sel = restacked_ds.interactive(loc="left").sel(**xs_1D).squeeze(drop=True)

In [ ]:
rds_sel["R"].sel(delta=slice(0.01, 0.15)).hvplot(x="wavelength", by="delta")*rds_sel["R"].sel(delta=0).hvplot(x="wavelength", line_dash="dashed", color="black", line_width=2)

In [ ]:
rds_sel["R"].hvplot(x="wavelength", y="delta")

<!-- Generally, the fewer layers, the less noise from disorder. -->

The noise is comparatively more important when the PBG is near the LO resonance.

In [ ]:
rds_sel_2D = rds_mean.interactive(loc="left").sel(delta=xs_1D["delta"], theta=xs_1D["theta"]).squeeze(drop=True)

In [ ]:
rds_sel_2D.R.hvplot(x="wavelength", by="run_bins", row="period", col="num_periods")

In [ ]:
# recipe for 1D comparison of disordered and periodic structure
var_key = "R"

overlay = (
    mean_and_std(restacked_ds, var_key=var_key)
    .squeeze()
    .hvplot(
        x="wavelength",
        y=f"{var_key}_mean",
        groupby=["delta", "num_periods", "period", "theta"],
        color=blue,
    )
    * mean_and_std(restacked_ds, var_key=var_key)
    .squeeze()
    .hvplot.area(
        x="wavelength",
        y=f"{var_key}_low",
        y2=f"{var_key}_high",
        groupby=["delta", "num_periods", "period", "theta"],
        alpha=0.2,
        color=blue,
    )
    * restacked_pds[var_key]
    .squeeze()
    .hvplot(
        x="wavelength",
        groupby=["num_periods", "period", "theta"],
        line_dash="dashed",
        color="black",
    )
    * max_min_pos(restacked_ds, var_key=var_key, dim="wavelength").hvplot(
        kind="scatter",
        x=f"{var_key}_max_wavelength",
        y=f"{var_key}_max",
        groupby=["delta", "num_periods", "period", "theta"],
        alpha=0.5,
        marker="x",
        color=blue,
    )
)

overlay

In [ ]:
Delta = 0.3 # Deltas: 0.1, 0.2, 0.3
N = 5
P = 200
theta = 0

fig = overlay.select(delta=Delta, num_periods=N, period=P, theta=theta).opts(
    opts.Curve(fontscale=2),
    # opts.Overlay(show_title=False),
    clone=True
)
display(fig)

In [ ]:
elt = 50
D = str(Delta).replace(".", "")

hv.save(
    fig,
    filename=f"data/run_{run_number}/figures/DLOPC_{var_key}_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}",
    fmt="png",
    toolbar=None,
)

In [ ]:
# recipe for 1D comparison of disordered and periodic structure
var_key = "R"

overlay = (
    mean_and_std(restacked_ds_05, var_key=var_key)
    .squeeze()
    .hvplot(
        x="wavelength",
        y=f"{var_key}_mean",
        groupby=["num_periods", "period", "theta"],
        color=blue,
    )
    * mean_and_std(restacked_ds_05, var_key=var_key)
    .squeeze()
    .hvplot.area(
        x="wavelength",
        y=f"{var_key}_low",
        y2=f"{var_key}_high",
        groupby=["num_periods", "period", "theta"],
        alpha=0.2,
        color=blue,
    )
    * restacked_pds[var_key]
    .squeeze()
    .hvplot(
        x="wavelength",
        groupby=["num_periods", "period", "theta"],
        line_dash="dashed",
        color="black"
    )
    * max_min_pos(restacked_ds_05, var_key=var_key, dim="wavelength").hvplot(
        kind="scatter",
        x=f"{var_key}_max_wavelength",
        y=f"{var_key}_max",
        groupby=["num_periods", "period", "theta"],
        alpha=0.5,
        marker="x",
        color=blue,
    )
)

overlay

In [ ]:
Delta = 0.5 # Deltas: 0.1, 0.2, 0.3
N = 50
P = 300 # Ps: 200, 250, 300
theta = 0

fig = overlay.select(num_periods=N, period=P, theta=theta).opts(
    opts.Curve(fontscale=2),
    # opts.Overlay(show_title=False),
    clone=True
)
display(fig)

In [ ]:
elt = 50
D = str(Delta).replace(".", "")

hv.save(
    fig,
    filename=f"data/run_{run_number}/figures/DLOPC_{var_key}_meancurve_wl_P{P}_elt{elt}_N{N}_th{theta}_D{D}",
    fmt="png",
    toolbar=None,
)

The clustering of scatter points around \~700 nm is very strange - seems attractive at low N but repulsive at high N.

In [ ]:
# recipe for 1D comparison of disordered and periodic structure
var_key = "T"

(
    mean_and_std(restacked_ds, var_key=var_key)
    .squeeze()
    .hvplot(
        x="wavelength",
        y=f"{var_key}_mean",
        groupby=["delta", "num_periods", "period", "theta"],
        color=yellow,
    )
    * mean_and_std(restacked_ds, var_key=var_key)
    .squeeze()
    .hvplot.area(
        x="wavelength",
        y=f"{var_key}_low",
        y2=f"{var_key}_high",
        groupby=["delta", "num_periods", "period", "theta"],
        alpha=0.2,
        color=yellow,
    )
    * restacked_pds[var_key]
    .squeeze()
    .hvplot(
        x="wavelength",
        groupby=["num_periods", "period", "theta"],
        line_dash="dashed",
        color="black",
    )
    * max_min_pos(restacked_ds, var_key=var_key, dim="wavelength").hvplot(
        kind="scatter",
        x=f"{var_key}_max_wavelength",
        y=f"{var_key}_max",
        groupby=["delta", "num_periods", "period", "theta"],
        alpha=0.5,
        marker="x",
        color=yellow,
    )
)

In [ ]:
# recipe for 1D comparison of disordered and periodic structure
var_key = "A"

(
    mean_and_std(restacked_ds, var_key=var_key)
    .squeeze()
    .hvplot(
        x="wavelength",
        y=f"{var_key}_mean",
        groupby=["delta", "num_periods", "period", "theta"],
        color=red,
    )
    * mean_and_std(restacked_ds, var_key=var_key)
    .squeeze()
    .hvplot.area(
        x="wavelength",
        y=f"{var_key}_low",
        y2=f"{var_key}_high",
        groupby=["delta", "num_periods", "period", "theta"],
        alpha=0.2,
        color=red,
    )
    * restacked_pds[var_key]
    .squeeze()
    .hvplot(
        x="wavelength",
        groupby=["num_periods", "period", "theta"],
        line_dash="dashed",
        color="black",
    )
    * max_min_pos(restacked_ds, var_key=var_key, dim="wavelength").hvplot(
        kind="scatter",
        x=f"{var_key}_max_wavelength",
        y=f"{var_key}_max",
        groupby=["delta", "num_periods", "period", "theta"],
        alpha=0.5,
        marker="x",
        color=red,
    )
)

In [ ]:
# recipe for 1D comparison of disordered and periodic structure
var_key = "A"

(
    mean_and_std(restacked_ds_05, var_key=var_key)
    .squeeze()
    .hvplot(
        x="wavelength",
        y=f"{var_key}_mean",
        groupby=["num_periods", "period", "theta"],
        color=red,
    )
    * mean_and_std(restacked_ds_05, var_key=var_key)
    .squeeze()
    .hvplot.area(
        x="wavelength",
        y=f"{var_key}_low",
        y2=f"{var_key}_high",
        groupby=["num_periods", "period", "theta"],
        alpha=0.2,
        color=red,
    )
    * restacked_pds[var_key]
    .squeeze()
    .hvplot(
        x="wavelength",
        groupby=["num_periods", "period", "theta"],
        line_dash="dashed",
        color="black",
    )
    * max_min_pos(restacked_ds_05, var_key=var_key, dim="wavelength").hvplot(
        kind="scatter",
        x=f"{var_key}_max_wavelength",
        y=f"{var_key}_max",
        groupby=["num_periods", "period", "theta"],
        alpha=0.5,
        marker="x",
        color=red,
    )
)

In [ ]:
# recipe for 1D comparison of disordered and periodic structure
var_key = "A"

(
    mean_and_std(restacked_norm_1, var_key=var_key)
    .squeeze()
    .hvplot(
        x="wavelength",
        y=f"{var_key}_mean",
        groupby=["delta", "num_periods", "period", "theta"],
        color=red,
    )
    * mean_and_std(restacked_norm_1, var_key=var_key)
    .squeeze()
    .hvplot.area(
        x="wavelength",
        y=f"{var_key}_low",
        y2=f"{var_key}_high",
        groupby=["delta", "num_periods", "period", "theta"],
        alpha=0.2,
        color=red,
    )
    * (
        enhancement_factor(
            ds=restacked_pds,
            ref=crs_1,
            common_dim="total_excitonic_thickness",
            method="groupby",
        )[var_key]
    )
    .squeeze()
    .hvplot(
        x="wavelength",
        groupby=["num_periods", "period", "theta"],
        line_dash="dashed",
        color="black",
    )
    * max_min_pos(restacked_norm_1, var_key=var_key, dim="wavelength").hvplot(
        kind="scatter",
        x=f"{var_key}_max_wavelength",
        y=f"{var_key}_max",
        groupby=["delta", "num_periods", "period", "theta"],
        alpha=0.5,
        marker="x",
        color=red,
    )
    * hv.HLine(1).opts(opts.HLine(line_dash="dotted", color="black"))
)

In [ ]:
# recipe for 1D comparison of disordered and periodic structure
var_key = "A"

(
    mean_and_std(restacked_norm_1_05, var_key=var_key)
    .squeeze()
    .hvplot(
        x="wavelength",
        y=f"{var_key}_mean",
        groupby=["num_periods", "period", "theta"],
        color=red,
    )
    * mean_and_std(restacked_norm_1_05, var_key=var_key)
    .squeeze()
    .hvplot.area(
        x="wavelength",
        y=f"{var_key}_low",
        y2=f"{var_key}_high",
        groupby=["num_periods", "period", "theta"],
        alpha=0.2,
        color=red,
    )
    * (
        enhancement_factor(
            ds=restacked_pds,
            ref=crs_1,
            common_dim="total_excitonic_thickness",
            method="groupby",
        )[var_key]
    )
    .squeeze()
    .hvplot(
        x="wavelength",
        groupby=["num_periods", "period", "theta"],
        line_dash="dashed",
        color="black",
    )
    * max_min_pos(restacked_norm_1_05, var_key=var_key, dim="wavelength").hvplot(
        kind="scatter",
        x=f"{var_key}_max_wavelength",
        y=f"{var_key}_max",
        groupby=["num_periods", "period", "theta"],
        alpha=0.5,
        marker="x",
        color=red,
    )
    * hv.HLine(1).opts(opts.HLine(line_dash="dotted", color="black"))
)

Disorder tends to flatten EF and reduce the max - so wavelength-specific enhancement should probably involve low disorder.

Overall absorptance EF seems lower in general with disorder, although I need to do an integrated EF to check.

In [ ]:
# recipe for 1D comparison of disordered and periodic structure
var_key = "R"

(
    (mean_and_std(restacked_ds, var_key=var_key) - restacked_pds[var_key])
    .squeeze()
    .hvplot(
        x="wavelength",
        y=f"{var_key}_mean",
        groupby=["delta", "num_periods", "period", "theta"],
    )
    * (mean_and_std(restacked_ds, var_key=var_key) - restacked_pds[var_key])
    .squeeze()
    .hvplot.area(
        x="wavelength",
        y=f"{var_key}_low",
        y2=f"{var_key}_high",
        groupby=["delta", "num_periods", "period", "theta"],
        alpha=0.2,
        color=blue
    )
    # * (-mean_and_std(restacked_ds, var_key=var_key))
    # .squeeze()
    # .hvplot.area(
    #     x="wavelength",
    #     y=f"{var_key}_std",
    #     groupby=["delta", "num_periods", "period", "theta"],
    #     alpha=0.2,
    #     color=blue
    # )
)

In [ ]:
# recipe for a nice statistical comparison
# var = ["Rs", "Rp", "R"]
# var = ["As", "Ap", "A"]
var = reflectance

(
    integrate_da(restacked_ds[var], dim="wavelength", normalisation=1).hvplot(
        kind="violin", groupby=["delta", "num_periods", "period", "theta"],
    )
    * integrate_da(restacked_pds[var], dim="wavelength", normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period", "theta"], legend=False
    )
)

In [ ]:
# recipe for a nice statistical comparison
# var = ["Rs", "Rp", "R"]
# var = ["As", "Ap", "A"]
# var = reflectance

layout = (
    integrate_da(restacked_ds_05[reflectance], dim="wavelength", normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period", "theta"], color=blue,
    )
    * integrate_da(restacked_pds[reflectance], dim="wavelength", normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period", "theta"], color=blue, legend=False
    )
    + integrate_da(restacked_ds_05[transmittance], dim="wavelength", normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period", "theta"], color=yellow,
    )
    * integrate_da(restacked_pds[transmittance], dim="wavelength", normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period", "theta"], color=yellow, legend=False
    )
    + integrate_da(restacked_ds_05[absorptance], dim="wavelength", normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period", "theta"], color=red,
    )
    * integrate_da(restacked_pds[absorptance], dim="wavelength", normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period", "theta"], color=red, legend=False
    )
).opts(opts.Layout(shared_axes=False)).cols(1)

layout

Notes (violin plots, integrated over wavelength but NOT theta):
- Integrated reflectance at normal incidence is *slightly lower* for disordered structures when N is low (\~<20) but *slightly higher* when N is high (\~>25)
    - It's quite surprising how little difference even high levels of disorder make to the integrated RTA as a whole
- When $\Delta$ is higher the threshold N seems to increase slightly, but the increased reflectance is also much stronger
    - At Delta=0.5 the threshold is around N=20 for P=200, 250 but N=40 for P=300
- The integrated reflectance is also higher at higher angles (for N=5, $\Delta$=0.4, look at $\theta$=35)
- The integrated absorptance is *generally* higher for the disordered structures, but not always


### Violin plots integrated over wavelength and theta

In [ ]:
# recipe for a nice statistical comparison
# var = ["Rs", "Rp", "R"]
# var = ["As", "Ap", "A"]
var = reflectance
theta_range = (0, 45)

(
    integrate_da(restacked_ds[var].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", groupby=["delta", "num_periods", "period"],
    )
    * integrate_da(restacked_pds[var].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period"], legend=False
    )
)

In [ ]:
# integrated over theta
theta_range = (0, 45)

layout = (
    integrate_da(restacked_ds_05[reflectance].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period"], color=blue,
    )
    * integrate_da(restacked_pds[reflectance].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period"], color=blue, legend=False
    )
    + integrate_da(restacked_ds_05[transmittance].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period"], color=yellow,
    )
    * integrate_da(restacked_pds[transmittance].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period"], color=yellow, legend=False
    )
    + integrate_da(restacked_ds_05[absorptance].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period"], color=red,
    )
    * integrate_da(restacked_pds[absorptance].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period"], color=red, legend=False
    )
).opts(opts.Layout(shared_axes=False)).cols(1)

layout

In [ ]:
N = 5
P = 250
elt = 50
fig = layout.select(num_periods=N, period=P).opts(
    opts.Violin(xlabel="", fontscale=2),
    opts.Layout(show_title=False),
    clone=True
)
display(fig)

In [ ]:
for f, label in zip(fig, ["R", "T", "A"]):
    hv.save(f, filename=f"data/run_{run_number}/figures/DLOPC_{label}_violin_wl_th_P{P}_elt{elt}_N{N}_wltot_th{theta_range[0]}-{theta_range[1]}", fmt="png", toolbar=None)

In [ ]:
N = 20
P = 250
elt = 50
fig = layout.select(num_periods=N, period=P).opts(
    opts.Violin(xlabel="", fontscale=2),
    opts.Layout(show_title=False),
    clone=True
)
display(fig)

In [ ]:
for f, label in zip(fig, ["R", "T", "A"]):
    hv.save(f, filename=f"data/run_{run_number}/figures/DLOPC_{label}_violin_wl_th_P{P}_elt{elt}_N{N}_wltot_th{theta_range[0]}-{theta_range[1]}", fmt="png", toolbar=None)

In [ ]:
N = 50
P = 250
elt = 50
fig = layout.select(num_periods=N, period=P).opts(
    opts.Violin(xlabel="", fontscale=2),
    opts.Layout(show_title=False),
    clone=True
)
display(fig)

In [ ]:
for f, label in zip(fig, ["R", "T", "A"]):
    hv.save(f, filename=f"data/run_{run_number}/figures/DLOPC_{label}_violin_wl_th_P{P}_elt{elt}_N{N}_wltot_th{theta_range[0]}-{theta_range[1]}", fmt="png", toolbar=None)

Just RTA plots

In [ ]:

layout = (
    integrate_da(restacked_ds_05["R"], dim=["wavelength"], normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period", "theta"], color=blue,
    )
    * integrate_da(restacked_pds["R"], dim=["wavelength"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period", "theta"], color=blue, legend=False
    )
    + integrate_da(restacked_ds_05["T"], dim=["wavelength"], normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period", "theta"], color=yellow,
    )
    * integrate_da(restacked_pds["T"], dim=["wavelength"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period", "theta"], color=yellow, legend=False
    )
    + integrate_da(restacked_ds_05["A"], dim=["wavelength"], normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period", "theta"], color=red,
    )
    * integrate_da(restacked_pds["A"], dim=["wavelength"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period", "theta"], color=red, legend=False
    )
).opts(opts.Violin(width=200), opts.Layout(shared_axes=False)).cols(3)

layout

In [ ]:
N = 50
P = 250
theta = 0
elt = 50
fig = layout.select(num_periods=N, period=P, theta=theta).opts(
    opts.Violin(xlabel="", fontscale=2, width=200, height=300),
    opts.Layout(show_title=False),
    clone=True
)
display(fig)

In [ ]:
hv.save(
    fig,
    filename=f"data/run_{run_number}/figures/DLOPC_RTA_violin_wl_P{P}_elt{elt}_N{N}_wltot_th{theta}",
    fmt="png",
    toolbar=None,
)

In [ ]:
# integrated over theta
theta_range = (0, 45)

layout = (
    integrate_da(restacked_ds_05["R"].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period"], color=blue,
    )
    * integrate_da(restacked_pds["R"].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period"], color=blue, legend=False
    )
    + integrate_da(restacked_ds_05["T"].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period"], color=yellow,
    )
    * integrate_da(restacked_pds["T"].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period"], color=yellow, legend=False
    )
    + integrate_da(restacked_ds_05["A"].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", groupby=["num_periods", "period"], color=red,
    )
    * integrate_da(restacked_pds["A"].sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", groupby=["num_periods", "period"], color=red, legend=False
    )
).opts(opts.Violin(width=200), opts.Layout(shared_axes=False)).cols(3)

layout

In [ ]:
N = 50
P = 250
elt = 50
fig = layout.select(num_periods=N, period=P).opts(
    opts.Violin(xlabel="", fontscale=2, width=200, height=300),
    opts.Layout(show_title=False),
    clone=True
)
display(fig)

Not convinced these are good plots

In [ ]:
hv.save(
    fig,
    filename=f"data/run_{run_number}/figures/DLOPC_RTA_violin_wl_th_P{P}_elt{elt}_N{N}_wltot_th{theta_range[0]}-{theta_range[1]}",
    fmt="png",
    toolbar=None,
)

In [ ]:
# integrated over theta
var = "R"
theta_range = (0, 45)
Delta = 0.1
N = [5, 20, 50]
P = 300  # Ps: 200, 250, 300

layout = (
    integrate_da(restacked_ds[var].sel(delta=Delta).sel(period=P).sel(num_periods=N).sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", row="num_periods", color=blue,
    )
    * integrate_da(restacked_pds[var].sel(period=P).sel(num_periods=N).sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", row="num_periods", color=blue, legend=False
    )
).opts(
    # opts.Violin(width=500),
    # opts.BoxWhisker(width=500),
    # opts.GridSpace(frame_width=500),  # frame_width is broken, need new Holoviews version (see https://github.com/holoviz/holoviews/issues/4051)
    opts.GridSpace(plot_size=(200, 300))
)

layout

In [ ]:
# # sanity check - is the minimum integrated R < 0, as implied by the leftmost plot?
# # no, it's an artifact of the violin plotting the probability distribution
# integrate_da(
#     restacked_ds_05["R"]
#     .sel(period=P)
#     .sel(num_periods=5)
#     .sel(theta=slice(*theta_range)),
#     dim=["wavelength", "theta"],
#     normalisation=1,
# ).min("run").compute()

In [ ]:
elt = 50
fig = layout.opts(
    opts.Violin(
        xlabel="",
        xaxis=None,
        fontscale=2,
    ),
    opts.GridSpace(
        fontscale=2,
        toolbar=None,
        # xaxis=None
    ),
    clone=True,
)
display(fig)

In [ ]:
Ns = "-".join([str(i) for i in N])
ths = "-".join([str(i) for i in theta_range])
D = str(Delta).replace(".", "")
hv.save(
    fig,
    filename=f"data/run_{run_number}/figures/DLOPC_{var}_violin_wl_th_P{P}_elt{elt}_N{Ns}_wltot_th{ths}_D{D}",
    fmt="png",
    toolbar=None,
)

In [ ]:
# integrated over theta
var = "R"
theta_range = (0, 45)
N = [5, 20, 50]
P = 300  # Ps: 200, 250, 300

layout = (
    integrate_da(restacked_ds_05[var].sel(period=P).sel(num_periods=N).sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", row="num_periods", color=blue,
    )
    * integrate_da(restacked_pds[var].sel(period=P).sel(num_periods=N).sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", row="num_periods", color=blue, legend=False
    )
).opts(
    # opts.Violin(width=500),
    # opts.BoxWhisker(width=500),
    # opts.GridSpace(frame_width=500),  # frame_width is broken, need new Holoviews version (see https://github.com/holoviz/holoviews/issues/4051)
    opts.GridSpace(plot_size=(200, 300))
)

layout

In [ ]:
# # sanity check - is the minimum integrated R < 0, as implied by the leftmost plot?
# # no, it's an artifact of the violin plotting the probability distribution
# integrate_da(
#     restacked_ds_05["R"]
#     .sel(period=P)
#     .sel(num_periods=5)
#     .sel(theta=slice(*theta_range)),
#     dim=["wavelength", "theta"],
#     normalisation=1,
# ).min("run").compute()

In [ ]:
elt = 50
fig = layout.opts(
    opts.Violin(
        xlabel="",
        xaxis=None,
        fontscale=2,
    ),
    opts.GridSpace(fontscale=2, toolbar=None, xaxis=None),
    clone=True,
)
display(fig)

In [ ]:
Ns = "-".join([str(i) for i in N])
ths = "-".join([str(i) for i in theta_range])
hv.save(
    fig,
    filename=f"data/run_{run_number}/figures/DLOPC_{var}_violin_wl_th_P{P}_elt{elt}_N{Ns}_wltot_th{ths}",
    fmt="png",
    toolbar=None,
)

In [ ]:
# integrated over theta
var = "T"
theta_range = (0, 45)
N = [5, 20, 50]
P = 250

layout = (
    integrate_da(restacked_ds_05[var].sel(period=P).sel(num_periods=N).sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", row="num_periods", color=yellow,
    )
    * integrate_da(restacked_pds[var].sel(period=P).sel(num_periods=N).sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", row="num_periods", color=yellow, legend=False
    )
).opts(
    # opts.Violin(width=500),
    # opts.BoxWhisker(width=500),
    # opts.GridSpace(frame_width=500),  # frame_width is broken, need new Holoviews version (see https://github.com/holoviz/holoviews/issues/4051)
    opts.GridSpace(plot_size=(200, 300))
)

layout

In [ ]:
elt = 50
fig = layout.opts(
    opts.Violin(
        xlabel="",
        xaxis=None,
        ylim=(0.36, None),
        fontscale=2,
    ),
    opts.GridSpace(fontscale=2, toolbar=None, ),
    clone=True,
)
display(fig)

In [ ]:
Ns = "-".join([str(i) for i in N])
ths = "-".join([str(i) for i in theta_range])
hv.save(
    fig,
    filename=f"data/run_{run_number}/figures/DLOPC_{var}_violin_wl_th_P{P}_elt{elt}_N{Ns}_wltot_th{ths}",
    fmt="png",
    toolbar=None,
)

In [ ]:
# integrated over theta
var = "A"
theta_range = (0, 45)
N = [5, 20, 50]
P = 250

layout = (
    integrate_da(restacked_ds_05[var].sel(period=P).sel(num_periods=N).sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="violin", row="num_periods", color=red,
    )
    * integrate_da(restacked_pds[var].sel(period=P).sel(num_periods=N).sel(theta=slice(*theta_range)), dim=["wavelength", "theta"], normalisation=1).hvplot(
        kind="box", row="num_periods", color=red, legend=False
    )
).opts(
    # opts.Violin(width=500),
    # opts.BoxWhisker(width=500),
    # opts.GridSpace(frame_width=500),  # frame_width is broken, need new Holoviews version (see https://github.com/holoviz/holoviews/issues/4051)
    opts.GridSpace(plot_size=(200, 300))
)

layout

In [ ]:
elt = 50
fig = layout.opts(
    opts.Violin(
        xlabel="",
        xaxis=None,
        ylim=(0.08, 0.52),
        fontscale=2,
    ),
    opts.GridSpace(fontscale=2, toolbar=None, xaxis=None),
    clone=True,
)
display(fig)

In [ ]:
Delta = 0.5

In [ ]:
D = str(Delta).replace(".", "")

In [ ]:
Ns = "-".join([str(i) for i in N])
ths = "-".join([str(i) for i in theta_range])
hv.save(
    fig,
    filename=f"data/run_{run_number}/figures/DLOPC_{var}_violin_wl_th_P{P}_elt{elt}_N{Ns}_wltot_th{ths}_D{D}",
    fmt="png",
    toolbar=None,
)